In [ ]:
%pip install --upgrade numpy

In [ ]:
%pip install datasets

In [ ]:
%pip install -U transformers

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("mshenoda/spam-messages")
ds_train = ds["train"]
ds_test = ds["test"]
ds_val = ds["validation"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/804 [00:00<?, ?B/s]

spam_messages_train.csv:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

spam_messages_val.csv: 0.00B [00:00, ?B/s]

spam_messages_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/47392 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5923 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5926 [00:00<?, ? examples/s]

In [ ]:
from collections import Counter
import numpy as np
def del_undersample(ds):
    counts = Counter(ds["label"])
    min_count = min(counts.values())

    balanced_indices = []
    rng = np.random.default_rng(42)

    for label in counts:
        indices = [i for i, l in enumerate(ds["label"]) if l == label]
        chosen = rng.choice(indices, size=min_count, replace=False)
        balanced_indices.extend(chosen)

    ds = ds.select(balanced_indices)
    ds = ds.shuffle(seed=42)
    return ds

In [ ]:
unique_labels = set(ds_train['label'])
label2id = {v: k for k, v in enumerate(sorted(unique_labels))}

def encode_labels(example):
    return {'labels': label2id[example['label']]}

In [ ]:
ds_train = del_undersample(ds_train)
ds_train = ds_train.map(encode_labels)

ds_val = del_undersample(ds_val)
ds_val = ds_val.map(encode_labels)

ds_test = del_undersample(ds_test)
ds_test = ds_test.map(encode_labels)

Map:   0%|          | 0/37664 [00:00<?, ? examples/s]

Map:   0%|          | 0/4894 [00:00<?, ? examples/s]

Map:   0%|          | 0/4662 [00:00<?, ? examples/s]

In [ ]:
%pip install torch torchvision

In [ ]:
%pip install --upgrade numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%pip install 'accelerate>=0.26.0'

In [ ]:
train_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    optim="adamw_torch"
)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=512)

encoded_ds_train = ds_train.map(preprocess_function, batched=True)
encoded_ds_train = encoded_ds_train.remove_columns(["text", 'label'])

encoded_ds_val = ds_val.map(preprocess_function, batched=True)
encoded_ds_val = encoded_ds_val.remove_columns(["text", 'label'])

encoded_ds_test = ds_test.map(preprocess_function, batched=True)
encoded_ds_test = encoded_ds_test.remove_columns(["text", 'label'])


Map:   0%|          | 0/37664 [00:00<?, ? examples/s]

Map:   0%|          | 0/4894 [00:00<?, ? examples/s]

Map:   0%|          | 0/4662 [00:00<?, ? examples/s]

In [ ]:
%pip install scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds),
        "auc": roc_auc_score(p.label_ids, preds),
        "recall": recall_score(p.label_ids, preds),
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_ds_train,
    eval_dataset=encoded_ds_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2703164887.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print(encoded_ds_train[0])

{'labels': 1, 'input_ids': [101, 2129, 2024, 8038, 1012, 1012, 1012, 1012, 2188, 12384, 2121, 2017, 2031, 2042, 3653, 1011, 4844, 2005, 1037, 1002, 4278, 1010, 2199, 2188, 5414, 2012, 1037, 1017, 1012, 2423, 1003, 4964, 3446, 1012, 2023, 3749, 2003, 2108, 3668, 2000, 2017, 4895, 8663, 27064, 2135, 1998, 2115, 4923, 2003, 1999, 2053, 2126, 1037, 5387, 1012, 2000, 2202, 5056, 1997, 2023, 3132, 2051, 4495, 2035, 2057, 3198, 2003, 2008, 2017, 3942, 2256, 4037, 1998, 3143, 1996, 1015, 3371, 2695, 6226, 2433, 2172, 3112, 1010, 3533, 11338, 7011, 17101, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc,Recall
1,0.078500,0.066438,0.981610,0.981737,0.981610,0.988557
2,0.029700,0.070444,0.984879,0.984836,0.984879,0.982019
3,0.010400,0.077977,0.986718,0.986689,0.986718,0.984471


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=7062, training_loss=0.051177212091216076, metrics={'train_runtime': 1919.0917, 'train_samples_per_second': 58.878, 'train_steps_per_second': 3.68, 'total_flos': 2.972944436723712e+16, 'train_loss': 0.051177212091216076, 'epoch': 3.0})